In [17]:
# Here define data directory and path to metadata csv file
metadata_file='/nfs/zorba/DOWNLOADED/IXI/IXI.xls'
metadata_sheet='Table'
data_path='/nfs/zorba/DOWNLOADED/IXI/data/'
glob_exp='*.nii.gz'


In [20]:
# Output of this cell will be scans_indexed (list of scans that have metadata) and df_indexed 
# (metadata for subjects that have available scans)

# Import IXI spreadsheet, df_* contains the information from the demographics spreadsheet
import pandas as pd
import glob
import os
workbook = pd.ExcelFile(metadata_file) 
df=workbook.parse(metadata_sheet)

# Generate list of relative paths to the IXI T1 scans. scans_* contains information from the downloaded files
os.chdir(data_path)
scans=sorted(glob.glob(glob_exp))

# Extract the subject ids and scanner ids from the file names
# (scanner id is not listed in the spreadsheet so need to extract from filenames in this case)
scans_scanner_ids=[] 
scans_subject_ids=[]
for s in range(0,len(scans)):
    tmp=scans[s]
    
    # get scanner id by grabbing letters in between first two dashes
    first_dash=tmp.find('-')
    second_dash=tmp.find('-',7)
    scans_scanner_ids.append(tmp[first_dash+1:second_dash])
    
    # get subject id
    scans_subject_ids.append(tmp[0:6])

# Determine the indices of the subject ids in the workbook corresponding to
# the available scans. df_subject_ids from spreadsheet, scans_subject_ids from glob
# There may be way to do this easier with pandas later
# First create list of subject_ids from the df
df_subject_ids=df['IXI_ID'].tolist()

# Turn number into 3 digit string preceded by "IXI"
df_subject_ids=['IXI' + str(int(x)).zfill(3) for x in df_subject_ids]

# Here get just the list of subjects with actual files
# First get the instersect of subject ids from metadata file and list of scans
inter = set(df_subject_ids).intersection(scans_subject_ids)

# Get lists of intersection indeces for df and scans
df_idx=[];
scans_idx=[];
for el in inter:
    df_idx.append(df_subject_ids.index(el))
    scans_idx.append(scans_subject_ids.index(el))
    
# Here get indeces of inter in df_subjects_ids
# scans_scanner_ids_indexed=[]; scans_indexed=[]; 

# Use ind to reindex the df and scans list
df_indexed=df.loc[df_idx]

#T = [L[i] for i in Idx]
scans_scanner_ids_indexed=[scans_scanner_ids[i] for i in scans_idx]
scans_indexed=[scans[i] for i in scans_idx]

print scans_indexed
print df_indexed['IXI_ID']
    

['IXI578-Guys-1078-T1.nii.gz', 'IXI579-Guys-1126-T1.nii.gz', 'IXI576-Guys-1077-T1.nii.gz', 'IXI577-HH-2661-T1.nii.gz', 'IXI574-IOP-1156-T1.nii.gz', 'IXI575-HH-2658-T1.nii.gz', 'IXI572-HH-2605-T1.nii.gz', 'IXI573-IOP-1155-T1.nii.gz', 'IXI571-IOP-1154-T1.nii.gz', 'IXI508-HH-2268-T1.nii.gz', 'IXI503-Guys-1021-T1.nii.gz', 'IXI502-Guys-1020-T1.nii.gz', 'IXI501-Guys-1016-T1.nii.gz', 'IXI500-Guys-1017-T1.nii.gz', 'IXI507-Guys-1036-T1.nii.gz', 'IXI506-Guys-1035-T1.nii.gz', 'IXI505-Guys-1026-T1.nii.gz', 'IXI504-Guys-1025-T1.nii.gz', 'IXI588-IOP-1158-T1.nii.gz', 'IXI531-Guys-1057-T1.nii.gz', 'IXI488-Guys-1015-T1.nii.gz', 'IXI489-Guys-1014-T1.nii.gz', 'IXI482-HH-2178-T1.nii.gz', 'IXI483-HH-2177-T1.nii.gz', 'IXI480-Guys-1033-T1.nii.gz', 'IXI481-HH-2175-T1.nii.gz', 'IXI486-Guys-1005-T1.nii.gz', 'IXI487-Guys-1037-T1.nii.gz', 'IXI484-HH-2179-T1.nii.gz', 'IXI485-HH-2180-T1.nii.gz', 'IXI477-IOP-1141-T1.nii.gz', 'IXI476-IOP-1140-T1.nii.gz', 'IXI475-IOP-1139-T1.nii.gz', 'IXI474-IOP-1138-T1.nii.gz', 'IXI3

In [25]:
# Set up connection details
import pyxnat
import os

# Set project name
project_name="IXI"

# connect to XNAT instance
xnat=pyxnat.Interface(config='/home/spantazatos/xnat_sp.cfg')

# Create the project in XNAT
# First check if it already exists
xnat.select.project(project_name).create()

# Iterate over each subject in scans (or df_sorted)
# Insert T1 scan, 
for sub in range(len(df_sorted)):
    print data_path + scans[sub]
    
    row = next(df_indexed.iterrows())[1]
    print row
    #print df_sorted["IXI_ID"][sub]
    #xnat.select.project(project_name).subject('demoSubj_1').create()

print xnat.select.projects().get()

#xnat.select.project('demoProj_1').accessibility()
#xnat.select.project('demoProj_1').set_accessibility(accessibility='private')


#db.select.project('IXI_TEST').create()
print "ok"

/nfs/zorba/DOWNLOADED/IXI/data/IXI002-Guys-0828-T1.nii.gz
552
/nfs/zorba/DOWNLOADED/IXI/data/IXI012-HH-1211-T1.nii.gz
IXI_ID                               578
SEX_ID (1=m, 2=f)                      1
HEIGHT                               177
WEIGHT                               100
ETHNIC_ID                              1
MARITAL_ID                             2
OCCUPATION_ID                          1
QUALIFICATION_ID                       5
DOB                           1948-11-24
DATE_AVAILABLE                         1
STUDY_DATE           2006-10-06 00:00:00
AGE                             57.86448
Name: 552, dtype: object
/nfs/zorba/DOWNLOADED/IXI/data/IXI013-HH-1212-T1.nii.gz


IndexError: tuple index out of range

In [15]:
import pandas as pd
x1=pd.Index(['a','b','c','d'])
x2=pd.Index(['a','c'])
inter=pd.Index.intersection(x1,x2).tolist()
print inter
# Here retrieve original indeces
x1_idx=['a','b','c','d'].index('c')
print x1_idx

['a', 'c']
2
